In [1]:
from mxnet import gluon
from mxnet import ndarray as nd
from mxnet.gluon import nn
import mxnet

In [2]:
mxnet.__version__

'1.1.0'

In [3]:
class tiny_yolo(nn.Block):
    def __init__(self,num_classes,verbose = False,**kwargs):
        super(tiny_yolo,self).__init__(**kwargs)
        self.verbose = verbose
        self.net = nn.Sequential()
        with self.name_scope():
            #0-1 layer
            net1 = nn.Sequential()
            net1.add(
                nn.Conv2D(16,kernel_size=3,padding=1),
                nn.BatchNorm(),
                nn.Activation('relu'),
                nn.MaxPool2D(pool_size=2,strides=2)
            )
            #2-3 layer
            net2 = nn.Sequential()
            net2.add(
                nn.Conv2D(32,kernel_size=3,padding=1),
                nn.BatchNorm(),
                nn.Activation('relu'),
                nn.MaxPool2D(pool_size=2,strides=2)
            )
            #4-5 layer
            net3 = nn.Sequential()
            net3.add(
                nn.Conv2D(64,kernel_size=3,padding=1),
                nn.BatchNorm(),
                nn.Activation('relu'),
                nn.MaxPool2D(pool_size=2,strides=2)
            )
            #6-7 layer
            net4 = nn.Sequential()
            net4.add(
                nn.Conv2D(128,kernel_size=3,padding=1),
                nn.BatchNorm(),
                nn.Activation('relu'),
                nn.MaxPool2D(pool_size=2,strides=2)
            )
            #8-9 layer
            net5 = nn.Sequential()
            net5.add(
                nn.Conv2D(256,kernel_size=3,padding=1),
                nn.BatchNorm(),
                nn.Activation('relu'),
                nn.MaxPool2D(pool_size=2,strides=2)
            )
            #10-11 layer
            net6 = nn.Sequential()
            net6.add(
                nn.Conv2D(512,kernel_size=3,padding=1),
                nn.BatchNorm(),
                nn.Activation('relu'),
                nn.MaxPool2D(pool_size=2,strides=1)
            )
            #12-14
            net7 = nn.Sequential()
            net7.add(
                nn.Conv2D(1024,kernel_size=3,padding=1,activation='relu'),
                nn.BatchNorm(),
                nn.Conv2D(512,kernel_size=3,padding=1,activation='relu'),
                nn.BatchNorm(),
#                 nn.Conv2D(5*(num_classes+5),kernel_size=1)
                nn.Conv2D(num_classes,kernel_size=1)
#                 nn.Dense(5*(num_classes+5))
            )
            self.net = nn.Sequential()
            self.net.add(net1,net2,net3,net4,net5,net6,net7)
    
    def forward(self,x):
        out = x
        for i,b in enumerate(self.net):
            out = b(out)
            if self.verbose:
                print('block %d shape: %s'%(i+1,out.shape))
        return out
            

In [4]:
net = tiny_yolo(3,True)
net.initialize()
x = nd.random.uniform(shape=(4,3,416,416))
y = net(x)
y.shape

block 1 shape: (4, 16, 208, 208)
block 2 shape: (4, 32, 104, 104)
block 3 shape: (4, 64, 52, 52)
block 4 shape: (4, 128, 26, 26)
block 5 shape: (4, 256, 13, 13)
block 6 shape: (4, 512, 12, 12)
block 7 shape: (4, 3, 12, 12)


(4, 3, 12, 12)

In [5]:
import utils1
from mxnet import init
train_data, test_data = utils1.load_data_fashion_mnist(batch_size=16,resize=96)
ctx = utils1.try_gpu()
# ctx
net = tiny_yolo(10)
net.initialize(ctx=ctx,init=init.Xavier())
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.03})
# trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.05})
# train_data.dataset[:]



In [6]:
# utils1.train(train_data,test_data,net,loss,trainer,ctx,num_epochs=2)

In [7]:
class mydense(nn.Block):
    def __init__(self,units,in_units,**kwargs):
        super(mydense,self).__init__(**kwargs)
        with self.name_scope():
            self.weight = self.params.get('weight',shape=(in_units,units))
            self.bais = self.params.get('bais',shape=(units,))
    def forward(self,x):
        linear = nd.dot(x,self.weight.data())+self.bais.data()
        return nd.relu(linear)
            

In [8]:
dense = mydense(5, in_units=10, prefix='o_my_dense_')
dense.params


o_my_dense_ (
  Parameter o_my_dense_weight (shape=(10, 5), dtype=<class 'numpy.float32'>)
  Parameter o_my_dense_bais (shape=(5,), dtype=<class 'numpy.float32'>)
)

In [13]:
x = nd.arange(9).reshape((3,3))
s = x.T*x
s


[[  0.   3.  12.]
 [  3.  16.  35.]
 [ 12.  35.  64.]]
<NDArray 3x3 @cpu(0)>

In [54]:
snp = s.asnumpy()
snp



array([[  0.,   3.,  12.],
       [  3.,  16.,  35.],
       [ 12.,  35.,  64.]], dtype=float32)

In [69]:
import numpy as np
spinv = np.linalg.pinv(snp)
xs = nd.array(spinv)*x.T                
xs


[[ 0.         -1.89999986  1.44999993]
 [-0.63333333  1.59999967 -0.69999993]
 [ 0.48333332 -0.49999988  0.19999997]]
<NDArray 3x3 @cpu(0)>

In [72]:
x = nd.arange(48).reshape((1,1,8,6))
x


[[[[  0.   1.   2.   3.   4.   5.]
   [  6.   7.   8.   9.  10.  11.]
   [ 12.  13.  14.  15.  16.  17.]
   [ 18.  19.  20.  21.  22.  23.]
   [ 24.  25.  26.  27.  28.  29.]
   [ 30.  31.  32.  33.  34.  35.]
   [ 36.  37.  38.  39.  40.  41.]
   [ 42.  43.  44.  45.  46.  47.]]]]
<NDArray 1x1x8x6 @cpu(0)>

In [91]:
y = nd.array([[0,1,1,4,4]])
nd.ROIPooling(x,y,pooled_size=(2,2),spatial_scale=1)


[[[[ 14.  16.]
   [ 26.  28.]]]]
<NDArray 1x1x2x2 @cpu(0)>